In [1]:
import torch
from transformers import AutoModelForImageTextToText, AutoProcessor, BitsAndBytesConfig
from huggingface_hub import login
import textwrap

In [ ]:
# 1. Login และตั้งค่า Model
HF_TOKEN = "ใส่ Access Token ของ gemma" #ต้องกด Accept Licencse จาก Hugging Face ก่อน
login(token=HF_TOKEN)

In [3]:
# ตรวจสอบว่ามองเห็น GPU หรือไม่
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"กำลังทำงานบน: {device.upper()}")

กำลังทำงานบน: CUDA


In [4]:
# 2. ตั้งค่าการโหลดโมเดล (Force GPU + Optimization)
# หาก GPU ของคุณมีแรม (VRAM) น้อยกว่า 16GB แนะนำให้เปิดใช้งาน 4-bit ด้านล่างนี้
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [5]:
model_id = "google/gemma-3-4b-it"
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    device_map={"": 0}, # บังคับให้โหลดลง GPU ตัวแรก
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
def translate_with_context(long_text, chunk_size=2000):
    """
    long_text: ข้อความภาษาอังกฤษยาวๆ
    chunk_size: จำนวนตัวอักษรต่อรอบ (ปรับได้ตามความเหมาะสม)
    """
    # แบ่งข้อความออกเป็นส่วนๆ
    import textwrap
    chunks = textwrap.wrap(long_text, width=chunk_size, break_long_words=False, replace_whitespace=False)
    
    full_translation = []
    # สร้าง 'memory' เก็บประวัติการแปล (System Instruction + Previous Work)
    chat_history = [
        {
            "role": "user",
            "content": "You are a professional translator. Translate English to Thai naturally. Keep the tone consistent and remember specialized terms used in previous parts."
        },
        {
            "role": "assistant",
            "content": "I understand. I will maintain consistency and natural Thai flow throughout the translation. Please provide the first part."
        }
    ]

    for i, chunk in enumerate(chunks):
        print(f"--- Translating Part {i+1}/{len(chunks)} ---")
        
        # เพิ่มข้อความส่วนใหม่เข้าไปใน history
        chat_history.append({
            "role": "user", 
            "content": f"Translate this part. Maintain consistency with previous parts if any:\n\n{chunk}"
        })

        # เตรียม Input โดยใช้ Chat Template (ซึ่งจะรวมประวัติทั้งหมด)
        prompt = processor.apply_chat_template(chat_history, add_generation_prompt=True, tokenize=False)
        inputs = processor(text=prompt, return_tensors="pt").to(model.device)

        # Generate คำแปล
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=2048, # ปรับตามความเหมาะสมของแต่ละ Chunk
                do_sample=True,
                temperature=0.2, # ใช้ค่าต่ำเพื่อให้แปลแม่นยำ
                top_p=0.9
            )

        # แกะเฉพาะคำตอบล่าสุดออกมา
        response = processor.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        
        print(f"Done part {i+1}")
        full_translation.append(response)

        # เพิ่มคำแปลเข้าไปใน history เพื่อให้รอบหน้าโมเดลเห็นว่า 'แปลอะไรไปแล้ว'
        chat_history.append({"role": "assistant", "content": response})

        # (Optional) ป้องกันประวัติยาวเกินไปจนแรมเต็ม 
        # แม้ Gemma 3 รับได้ 128K แต่ถ้า VRAM คุณน้อย อาจต้องเก็บแค่ 3-5 ส่วนล่าสุด
        if len(chat_history) > 10: 
            chat_history = [chat_history[0], chat_history[1]] + chat_history[-6:]

    return "\n".join(full_translation)

In [7]:
LN_part2 = """Chapter 1: Ais SOS
Loki Familia's expedition has failed, and despair filled the air as reports confirmed their devastating loss on the 60th Floor. Adventurers and observers alike were struck with shock and terror as the news spread throughout the city, overwhelming them with grief and disbelief. The sight of fallen comrades, bloodied and injured, painted a grim picture in the minds of those who witnessed it.

Amidst the chaos, Raul, clutching his injured arm, insisted that they must help their comrades still trapped in the Deep Floors. The realization that brave adventurers had not returned, including healers and support units, ignited anger and sorrow. The crowd reacted with cries of rage and sorrow, and many were unable to process the reality of the situation, crumbling under the weight of despair.

As chaos erupted in Orario, Bell realized that he couldn't find Ais among the returned, causing his heart to race, and despite knowing the circumstances, he shouted her name in desperation. Just as he prepared to run forward, Eina held him back, urging him to escape the suffocating chaos.

Despite the despair enveloping the city, Eina and the rest of the Guild staff tried to maintain order and care for those who did return. Bell, feeling overwhelmed, wrestled with his emotions as he witnessed Eina's own struggle with fear and sadness. He longed for the reassurance of his comrades, Finn and others, while battling the reality of their absence.

Leon placed a reassuring hand on Bell's shoulder, encouraging him to remain calm, impressing the weight of responsibility upon him. Eina remained a beacon of fragile strength, offering support despite her own terror; Bell felt a deep connection with her, echoing her apologies for the situation they faced.

As healers rushed in to assist those wounded, the scene grew increasingly somber. The cries for justice and anger from earlier echoed hollow in the face of the injured being carried away. Finally, Hestia, and the rest of Hestia Familia reunited with Bell, bringing a sense of urgency and concern amidst the unfolding disaster.

The survivors from the “Expedition” were taken to a special treatment facility run by the Dian Cecht Familia. In a hospital room were the Hestia Familia members, Leon, and Nina—except for Eina, who was on Guild duty. Lili looked tense, Mikoto kept her eyes lowered, and Haruhime tried to comfort them. Welf appeared calm next to Leon, though he was likely suppressing fear. Hestia’s mention of the "60th floor" made everyone uneasy.

Leon urged everyone to stay calm, but news from the 60th floor had us on edge. A monster had appeared, and Loki, the goddess of the Loki Familia, arrived looking distressed. She confirmed that the Dungeon Boss was gone and the “Deep Floors” had collapsed, leaving a powerful monster running wild below. Loki’s expression showed both anger and pain as she confirmed that her children were still trapped in the Dungeon—and that they needed to be rescued immediately.

Bell's heart raced with fear and hope when he realized that Ais and the others were still alive, as their blessings had not diminished. The other gods could sense their children through the divine blood linked to their Falna, which granted them those blessings. This realization reignited Bell's determination to help, and he urgently begged Loki to let him join the rescue team.

Loki seemed exasperated by his eagerness—though she had come to them seeking help. There was a moment of levity when Hestia and Syr playfully teased Loki, easing some of the tension in the room. Leon placed a comforting hand on Bell's shoulder, offering encouragement amidst the seriousness of the situation. Then Hedin asserted himself, announcing that he would lead the rescue operation as commander and that everyone would come under his command.

He had the experience needed to manage a large-scale mission, given his past leadership during the Familia War. Despite the heavy atmosphere, Bell felt honored to be among the first-class adventurers called upon for the mission. He asked Hedin what he could do to help, but to his surprise, Hedin told him to go to sleep. Bell was confused—how could he rest when the situation was so urgent?

Despite his protests, Hedin insisted that Bell recover from the fatigue he had built up during their recent trials in the Valley. Then, using magic, Hedin forcefully put Bell to sleep. Bell lost consciousness after being struck by Hedin's magic.

Bell collapsed onto the floor, prompting Hestia and Nina to scream in shock at the brutal act committed by Hedin. Stunned, Nina hesitated to help, but a golden-haired renard quickly cradled Bell’s head in her lap, while another girl began treating the burn injuries on his neck. The pallum and the elf criticized the excessive violence they had just witnessed.

The atmosphere was tense, with Hestia trembling beside them, caught between rage and restraint. Confusion and frustration hung thick in the air as Bell lay unconscious, receiving care from the girls—which annoyed Loki, due to Bell’s growing popularity among them. Mikoto tried to reassure Nina with a simple gesture, implying that experience made the difference in situations like this. Meanwhile, another girl watched silently, horrified by how normalized such violence had become in Bell’s life.

Hedin, seemingly unbothered by the unfolding chaos, turned to Leon and noted that he had also inhaled miasma from the Valley. Leon acknowledged this, adding that Nina’s purification magic had mostly protected them. Realizing this, Nina regained her composure.

Hedin went on to criticize Bell’s overreliance on his strength, explaining that he wouldn’t wake until he had fully recovered from exhaustion. The only reason he had forcefully put Bell to sleep in the first place was because he had noticed that Bell’s stamina recovery was slower than usual—something Bell himself had failed to recognize.

He emphasized that even high-level adventurers like Bell could misjudge their limits. Hestia gently lifted Bell onto her lap and applied a tranquilizing potion, the calming scent spreading through the room. She argued with Hedin, worried that if Bell fell too deeply into sleep, he might not wake up again—but he dismissed her concern.

Their argument was interrupted by Leon, who urged calmness. He questioned Hedin about the rising tensions, but Hedin stressed the importance of teamwork, especially since they had already suffered casualties. This revelation shocked Nina, while Leon’s expression hardened.

As tensions simmered, Hestia and Ryu noticed Syr quietly mourning the loss of [[Freya Familia]|her Familia members]. Syr revealed that three of them—Meluna, Restan, and Tanna—had perished during the expedition. They had been seasoned warriors who had fought alongside Bell and taught him valuable lessons.

Welf observed the somber mood, silently agreeing that keeping Bell asleep was the right choice. He was too kind-hearted to cope with such overwhelming sorrow. A heavy silence filled the room as the reality of loss settled over everyone.

Hedin, known for his strict nature, reluctantly acknowledged that the team had made the right call under the circumstances. Lili expressed disbelief at the Loki Familia’s forced retreat, highlighting the severity of the situation. Even adventurers like Finn, Riveria, Gareth, Tione, Tiona, Bete, Anakitty, Ais, Tsubaki, and Amid had not returned from the 60th floor. Their failure to escape suggested that even the strongest could be overwhelmed in the Dungeon’s deeper levels.

Syr then shared critical details: Ottar was still on the 49th floor, while Hogni had survived—but neither had returned. This news shocked Lili and the others, deepening the tension in the room. Questions arose about how Syr had access to such specific information, prompting Leon to ask about the source. Loki explained that the adventurers had been given an Eye Crystal to maintain communication until it shattered during the battle. Hestia confirmed that they had received this information directly from Loki.

A notable gap in knowledge had formed between those newly returned to the city and those who had already received updates. Discussions of a potential rescue mission began, with plans forming for immediate action. Mikoto, though visibly anxious, took a deep breath to steady herself. Welf presented a batch of Magic Swords imbued with elemental attributes. Hedin snatched the list from him without a word, signaling his approval of their readiness.

The Hestia Familia remained resolute in their decision to fight. Haruhime appeared emotional, while Lili stood pale and silent. Ryu quietly assessed the strength of their allies. Nina, sensing the group’s resolve, asked what their next steps were, emphasizing the importance of preparation for the Dungeon’s dangers.

She voiced concern over critical needs—water, food, and supplies—at such deep levels, reflecting on her own limited experience and feeling overwhelmed by the prospect of going deeper. She questioned how they would reach those trapped below, her doubt and worry surfacing. She also asked whether Lili and the others would be going, clearly worried about the number of adventurers capable of making the journey.

Abruptly, Hedin intervened, declaring that he would not leave any strength unused. He looked directly at Nina, his intense presence silencing her. He stated that Orario’s entire army would be mobilized for the mission, underscoring the gravity of the task ahead."""

In [8]:
# --- วิธีใช้งาน ---
english_document = LN_part2
result = translate_with_context(english_document)

# บันทึกลงไฟล์
with open("translated_thai.txt", "w", encoding="utf-8") as f:
    f.write(result)

--- Translating Part 1/5 ---
Done part 1
--- Translating Part 2/5 ---
Done part 2
--- Translating Part 3/5 ---
Done part 3
--- Translating Part 4/5 ---
Done part 4
--- Translating Part 5/5 ---
Done part 5
